In [ ]:
#import stuff
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext


# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("word count") \
      .getOrCreate() 


In [ ]:
#test if spark is running 
spark

In [ ]:
    ## tsv to dataFrame , for csv remove sep t , header option puts first line into header##
df= spark.read.option("sep","\t").csv('bands.tsv',header=True, inferSchema=True)
df.show()

In [ ]:
    ## merge df and df2 and clean

##prepare data##
df2= spark.read.option("sep","\t").csv('bands2.tsv',header=True, inferSchema=True)


    # union by name so it doesnt depend on position# 
#output= df.unionByName(df2, allowMissingColumns=True).dropDuplicates()

    #clean data of nulls
output = output.fillna(0)
output = output.na.fill("")
output = output.drop("name").withColumnRenamed("member","Name")
output.show()

In [ ]:
    # join to merge data for dependencies
output= df.join(df2,how="inner")
output = df.join(df2, on=['member','member']) 

In [ ]:
    ##write dataframe to file with tab  ##
output.write.option("header",True).option("delimiter","\t").csv("output/")

In [ ]:
    #ADD COLUMN with data
output.withColumn("im Number 5", lit(5)).show()

In [ ]:
## SPARK READ FILES AND USE SOME SQL Basics##
    #read local file
textFile = spark.read.text("new.txt")

    


## some interesting prompts for SQL
#
#
#x = {
#  size() counts the stuff inside
#  split(input,seperators) splits text at seperator
#  name("ColumName") rename colum
#  max()
#  min()
#  avg()
#  lower()
#  countDistinct()
#}
# .select([x])
# .agg(x) can be used to further use aggregations 
# .distinct(x)
# .collect() puts table into one line 
# .withColumnRenamed(from,to)

In [ ]:
    ## EXAMPLE GET SINGLE WORDS FROM FILE
dfText = spark.read.text("frankenstein.txt")

df = dfText.select(split(dfText.value,"[^a-zA-Z]").alias("word"))
df = df.select(explode(col("word")).alias("word"))
df = df.select(lower(col("word")).alias("word"))
df = df.select(regexp_extract(col("word"),"[a-z]{2,}|a|i",0).alias("word"))
df.show(50,False)

In [ ]:
# EXAMPLE GET MAX AMOUNT OF WORDS IN maxWords or in ONE LINE output #
maxWords=textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords")))
textFile.select(size(split(textFile.value, "\s+")).name("numWords")).agg(max(col("numWords"))).show(100,False)

In [ ]:
## negate search term ##
wordFilter = ["is","not","if","the"]
newFrame = df.filter(~col("word").isin(wordFilter))
newFrame.show()

In [ ]:
##Exploding list of words into ROWS##
from pyspark.sql.functions import explode, col
df = df.select(explode(col("Zeile")).alias("word"))

In [ ]:
##To lower case##
words_lower = words.select(lower(col("word")).alias("word_lower"))

In [ ]:
## remove empty string##
proper_words = words_clean.filter(col("cleanword") != "")
# or 
proper_words = words_clean.filter(~col("cleanword")== "")
## get word with 2 or more letters including a and i 
words_clean = words_lower.select(regexp_extract(col("word_lower"), "[a-z]{2+}|a|i", 0).alias("word"))


In [ ]:
##filter array##
exclDict = ["is", "not", "if", "the", "for", "of", "no", "at", "and"]
words_no_dict = words_nonull.where(~col("word").isin(exclDict))

In [ ]:
## COMPLEX EXAMPLE ##
esults = (
    spark.read.text("frankenstein.txt")
    .select(F.split(F.col("value"), " ").alias("line"))
    .select(F.explode(F.col("line")).alias("word"))
    .select(F.lower(F.col("word")).alias("word"))
    .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
    .where(F.col("word") != "")
    .groupby(F.col("word"))
    .count()
)

results.orderBy("count", ascending=False).show(10)
results.coalesce(1).write.csv("./results_single_partition.csv")